In [1]:
from sklearn.externals import joblib
from tensorflow.python.keras.models import load_model

In [2]:
import rospy
import numpy as np
import tensorflow as tf
from std_msgs.msg import Float32MultiArray
from std_msgs.msg import MultiArrayDimension
from std_msgs.msg import Float64

In [3]:
class PeePub:
    
    def __init__(self):
        self.session = tf.Session()
        self.graph = tf.get_default_graph()
        with self.graph.as_default():
            with self.session.as_default():
        
                scaler_filename = "scaler.save"
                self.scaler_model = joblib.load(scaler_filename) 
                self.model = load_model('test_model.h5')

                rospy.init_node('thermal_dnn', anonymous=False)
                rospy.Subscriber('/thermal_cam', Float32MultiArray, self.onThermalReading)
                self.pub = rospy.Publisher('pee', Float32MultiArray, queue_size=10)
                rospy.spin()
        
    def onThermalReading(self, readings):
        
        data = self.scaler_model.transform(np.array([readings.data]))
        pred = self.predict(data)
        
        msg = Float32MultiArray()
        msg.data = pred[0]
        self.pub.publish(msg)
        
#         ['dry','pee'][pred.argmax()]

    def predict(self, x):
        with self.graph.as_default():
            with self.session.as_default():
               y = self.model.predict(x)
        return y
        

In [ ]:
PeePub()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
[WARN] [1553920830.233946]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
